In [1]:
import cv2
from keras.utils import np_utils
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, Dense, Dropout, Activation, Flatten
from keras.models import Sequential
from keras.optimizers import Adam
from keras.layers.normalization import BatchNormalization

Using Theano backend.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [2]:
#import model
from keras.models import model_from_json
json_file=open('model.json','r')
loaded_model_json = json_file.read()
json_file.close()
model1= model_from_json(loaded_model_json)
model1.load_weights('model.h5')
model1.compile(loss='categorical_crossentropy',optimizer=Adam(),metrics=['accuracy'])
print('loaded')

loaded


In [3]:
img = cv2.imread("DevanagariHandwrittenCharacterDataset/Train/character_11_taamatar/214.png",cv2.IMREAD_GRAYSCALE)
import numpy as np
img=cv2.resize(img,(32,32))
#img=cv2.cvtColor(img,cv2.COLOR_RGB2BGR)
img=np.array(img)
img = img.reshape(-1,32,32,1)
#print(img)

In [4]:
print(model1.predict_classes(img))

[36]


In [5]:
import os
labels=[]
for folder in os.listdir("DevanagariHandwrittenCharacterDataset/Train"):
    labels.append(folder)
print(labels)  

['character_12_thaa', 'character_27_ra', 'character_13_daa', 'character_21_pa', 'character_14_dhaa', 'character_25_ma', 'character_30_motosaw', 'character_16_tabala', 'character_20_na', 'digit_8', 'digit_5', 'character_4_gha', 'character_32_patalosaw', 'character_18_da', 'character_9_jha', 'character_33_ha', 'digit_2', 'character_26_yaw', 'digit_7', 'digit_1', 'digit_4', 'character_31_petchiryakha', 'digit_9', 'character_3_ga', 'character_29_waw', 'character_6_cha', 'character_8_ja', 'character_23_ba', 'character_7_chha', 'character_28_la', 'character_22_pha', 'character_35_tra', 'digit_3', 'digit_6', 'character_34_chhya', 'character_24_bha', 'character_11_taamatar', 'digit_0', 'character_17_tha', 'character_10_yna', 'character_36_gya', 'character_19_dha', 'character_5_kna', 'character_2_kha', 'character_15_adna', 'character_1_ka']


In [6]:
index = model1.predict_classes(img)[0]
labels[index]

'character_11_taamatar'

In [7]:
import cv2
from keras.models import load_model
import numpy as np
from collections import deque

In [8]:
def main():
    cap = cv2.VideoCapture(0)
    Lower_green = np.array([110, 50, 50])
    Upper_green = np.array([130, 255, 255])
    pred_class=0
    pts = deque(maxlen=512)
    blackboard = np.zeros((480, 640, 3), dtype=np.uint8)
    digit = np.zeros((200, 200, 3), dtype=np.uint8)
    while (cap.isOpened()):
        ret, img = cap.read()
        img = cv2.flip(img, 1)
        imgHSV = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        mask = cv2.inRange(imgHSV, Lower_green, Upper_green)
        blur = cv2.medianBlur(mask, 15)
        blur = cv2.GaussianBlur(blur, (5, 5), 0)
        thresh = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
        cnts = cv2.findContours(thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)[1]
        center = None
        if len(cnts) >= 1:
            contour = max(cnts[0], key=cv2.contourArea)
            if cv2.contourArea(contour) > 250:
                ((x, y), radius) = cv2.minEnclosingCircle(contour)
                cv2.circle(img, (int(x), int(y)), int(radius), (0, 255, 255), 2)
                cv2.circle(img, center, 5, (0, 0, 255), -1)
                M = cv2.moments(contour)
                center = (int(M['m10'] / M['m00']), int(M['m01'] / M['m00']))
                pts.appendleft(center)
                for i in range(1, len(pts)):
                    if pts[i - 1] is None or pts[i] is None:
                        continue
                    cv2.line(blackboard, pts[i - 1], pts[i], (255, 255, 255), 10)
                    cv2.line(img, pts[i - 1], pts[i], (0, 0, 255), 5)
        elif len(cnts) == 0:
            if len(pts) != []:
                blackboard_gray = cv2.cvtColor(blackboard, cv2.COLOR_BGR2GRAY)
                blur1 = cv2.medianBlur(blackboard_gray, 15)
                blur1 = cv2.GaussianBlur(blur1, (5, 5), 0)
                thresh1 = cv2.threshold(blur1, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
                blackboard_cnts = cv2.findContours(thresh1.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)[1]
                if len(blackboard_cnts) >= 1:
                    cnt = max(blackboard_cnts, key=cv2.contourArea)
                    print(cv2.contourArea(cnt))
                    if cv2.contourArea(cnt) > 2000:
                        x, y, w, h = cv2.boundingRect(cnt)
                        digit = blackboard_gray[y:y + h, x:x + w]
                        # newImage = process_letter(digit)
                        pred_probab, pred_class = keras_predict(model1, digit)
                        print(pred_class, pred_probab)

            pts = deque(maxlen=512)
            blackboard = np.zeros((480, 640, 3), dtype=np.uint8)
        cv2.putText(img, "Conv Network :  " + str(labels[pred_class]), (10, 470),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.imshow("Frame", img)
        cv2.imshow("Contours", thresh)
        k = cv2.waitKey(10)
        if k == 27:
            cv2.destroyAllWindows()
            break

In [9]:
def keras_predict(model, image):
    processed = keras_process_image(image)
    print("processed: " + str(processed.shape))
    pred_probab = model.predict(processed)[0]
    pred_class = list(pred_probab).index(max(pred_probab))
    return max(pred_probab), pred_class

In [10]:
def keras_process_image(img):
    image_x = 32
    image_y = 32
    img = cv2.resize(img, (image_x, image_y))
    img = np.array(img, dtype=np.float32)
    img = np.reshape(img, (-1, image_x, image_y, 1))
    return img

In [11]:
keras_predict(model1, np.zeros((32, 32, 3), dtype=np.uint8))
main()

processed: (3, 32, 32, 1)


error: OpenCV(4.1.0) /io/opencv/modules/imgproc/src/shapedescr.cpp:274: error: (-215:Assertion failed) npoints >= 0 && (depth == CV_32F || depth == CV_32S) in function 'contourArea'
